### XGBoost

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import string
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from keybert import KeyBERT
from sentence_transformers import models, SentenceTransformer
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from imblearn.pipeline import Pipeline as ImbPipeline
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import clone

In [ ]:
descs = pd.read_csv("descs.csv")

In [ ]:
hscodes = pd.read_excel("hscodes.xlsx")

In [ ]:
df_xgb300 = pd.concat([hscodes,descs], axis=1)

In [ ]:
df_xgb300['Description'] = df_xgb300['Description'].astype(str).str.lower()
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hs code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hts code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hscode')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('h s code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hs-code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hs.code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('h.s.code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('htscode')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hts-code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hts.code')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: re.sub(r'\d{4}\.\d{2}','', x))
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: re.sub(r'\d{6}|\d{10}','', x))
df_xgb300['Description'] = df_xgb300['Description'].str.replace('\W', ' ')
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hts')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('hs')[0])
df_xgb300['Description'] = df_xgb300['Description'].apply(lambda x: x.split('h s')[0])

In [ ]:
# Download the set of stop words from NLTK
nltk.download('stopwords')

# Load stop words
stopwords = {word.lower() for word in stopwords.words('english')}

# Assuming you have a DataFrame 'df_xgb300' with a column 'text_column'
# ...

def remove_punctuation_and_stopwords(text):
    # Remove punctuation
    no_punct = "".join([char for char in text if char not in string.punctuation])

    # Remove stop words
    no_stopwords = " ".join([word for word in no_punct.split() if word not in stopwords])

    return no_stopwords

# Apply the function to your text column
df_xgb300['Description'] = df_xgb300['Description'].astype(str).apply(remove_punctuation_and_stopwords)

In [ ]:
# Group by the target column
grouped = df_xgb300.groupby('HS_Code')

# Filter groups with a size of at least 5
filtered_groups = grouped.filter(lambda x: len(x) >= 5)

In [ ]:
df_xgb300['HS_Code'] = df_sample4['HS_Code'].astype(int)

In [ ]:
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

# Initialize the KeyBERT model with the RoBERTa-based model
keybert_model = KeyBERT(model=model)

In [ ]:
def extract_keywords(text, num_keywords=10):
    return ' '.join([kw[0] for kw in keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), top_n=num_keywords)])

In [ ]:
kw_list =[]
for desc in df_xgb300['Description']:
    kw_list.append(extract_keywords(desc),50)

df_xgb300['Keywords'] = kw_list

In [ ]:
smote_100 = SMOTE(k_neighbors = 3)

In [ ]:
kw_list4 =[]
for desc in df_xgb300['Description']:
    kw_list4.append(extract_keywords(desc,50))

df_xgb300['Keywords'] = kw_list4

label_encoder = LabelEncoder()
df_xgb300['Labels_Enc'] = label_encoder.fit_transform(df_xgb300['HS_Code'])

In [ ]:
# Initialize the model
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', 
                                   num_class=len(np.unique(df_xgb300['HS_Code'])), 
                                   n_estimators  = 250, 
                                   max_depth = 20, 
                                   learning_rate = 0.2)
# Define Stratified K-Fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Initialize lists to store metrics
train_accuracies = []
accuracies = []
f1_scores = []

all_y_test = []
all_y_pred_probs = []

i=1

# Cross-validation loop
for train_index, test_index in stratified_kfold.split(df_xgb300['Keywords'], df_xgb300['Labels_Enc']):
    # Split data
    X_train, X_initial_test = df_xgb300.iloc[train_index]['Keywords'], df_xgb300.iloc[test_index]['Keywords']
    y_train, y_initial_test = df_xgb300.iloc[train_index]['Labels_Enc'], df_xgb300.iloc[test_index]['Labels_Enc']

    # Further split the initial test set into test and validation sets
    X_test, X_val, y_test, y_val = train_test_split(X_initial_test, y_initial_test, test_size=0.5, random_state=42)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    X_val_tfidf = vectorizer.transform(X_val)

    start_samp = time.time()
    X_train_smote, y_train_smote = smote_100.fit_resample(X_train_tfidf, y_train)
    end_samp = time.time()

    print(" Sampling time till 30: ", end_samp - start_samp)

    start_cv = time.time()
    # Train model
    xgb_classifier.fit(X_train_smote, y_train_smote, early_stopping_rounds=10, eval_set=[(X_val_tfidf, y_val)], eval_metric='mlogloss', verbose=True)
    end_cv = time.time()

    print("Fold ", i, "fitting time: ", end_cv - start_cv)

    i=i+1
    # Predict on test set
    y_pred = xgb_classifier.predict(X_test_tfidf)
    accuracies.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred, average='macro'))
    y_train_pred = xgb_classifier.predict(X_train_tfidf)
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    y_pred_probs = xgb_classifier.predict_proba(X_test_tfidf)
    all_y_pred_probs.append(y_pred_probs)
    all_y_test.extend(y_test)

print("Cross-validation time: ", end_cv - start_cv)

# Displaying average results
print(f"Average Accuracy: {np.mean(accuracies)}")
print(f"Average F1 Score: {np.mean(f1_scores)}")

print("Train accuracies: ",train_accuracies)
print("Test accuracies: ", accuracies)


In [ ]:
def top_n_accuracy(y_pred_probs, y_true, n):
    top_n_preds = np.argsort(y_pred_probs, axis=1)[:, -n:]
    return np.mean([y_true[i] in top_n_preds[i] for i in range(len(y_true))])

# Calculate and print top-N accuracies
for n in range(1, 10):
    accuracy = top_n_accuracy(y_pred_probs, y_test, n)
    print(f"Top-{n} Accuracy: {accuracy}")

### Stacking

In [ ]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression()))
    level0.append(('svm', SVC()))
    level0.append(('dt', DecisionTreeClassifier(random_state=42)))
	  # define meta learner model
    level1 = LogisticRegression()
	  # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['svm'] = SVC(class_weight = 'balanced', gamma = 0.1, kernel = 'rbf')
    models['dt'] = DecisionTreeClassifier(random_state=42)
    models['stacking'] = get_stacking()
    return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = StratifiedKFold(n_splits=5)
    scores = []
    
    for train_idx, test_idx in cv.split(X, y):
        # Split data
        X = X.astype(str)
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Apply TF-IDF
        vectorizer = TfidfVectorizer()
        print("TFIDF train fit start")
        X_train_tfidf = vectorizer.fit_transform(X_train)
        print("TFIDF train fit end")
        X_test_tfidf = vectorizer.transform(X_test)
        print("TFIDF test transform end")

        # Apply SMOTE
        smote = SMOTE(k_neighbors=3)
        X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

        # Fit and predict
        model_clone = clone(model)  # Clone the model to ensure independence between folds
        model_clone.fit(X_train_tfidf_smote, y_train_smote)
        y_pred = model_clone.predict(X_test_tfidf)

        # Evaluate
        score = accuracy_score(y_test, y_pred)
        scores.append(score)

        break

    return np.array(scores)


In [ ]:
# define dataset
X, y = df_xgb300['Keywords'], df_xgb300['Labels_Enc']
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()

In [ ]:
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print(name, mean(scores))